<a href="https://colab.research.google.com/github/MungaiMwangi001/LLMs/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load the dataset**

In [1]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("mteb/tweet_sentiment_extraction")
df = pd.DataFrame(dataset['train'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/240k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3432 [00:00<?, ? examples/s]

**TOkenizer**

In [2]:
from transformers import GPT2Tokenizer

# Loading the dataset to train our model
dataset = load_dataset("mteb/tweet_sentiment_extraction")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/26732 [00:00<?, ? examples/s]

Map:   0%|          | 0/3432 [00:00<?, ? examples/s]

**Initialize  the base model**

In [3]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

**Initialize the base model**

In [4]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
!pip install evaluate==0.4.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


**Evaluate Method**

**Finetune   usin the Trainer Method**

In [ ]:
import transformers
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import numpy as np
import torch
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
transformers.logging.set_verbosity_info()

print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")

# Calculate steps per epoch
n_gpu = torch.cuda.device_count() or 1
print(f"Using {n_gpu} GPU(s)")
total_batch_size = 1 * n_gpu * 4  # per_device_batch_size * n_gpu * grad_accumulation
steps_per_epoch = max(1, len(small_train_dataset) // total_batch_size)
print(f"Steps per epoch: {steps_per_epoch}")

# Define compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

# Training arguments compatible with v4.55.0
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    # Evaluation settings
    eval_strategy="steps", # Changed from evaluate_during_training
    eval_steps=steps_per_epoch,
    # Logging and saving
    logging_steps=100,
    save_steps=steps_per_epoch,
    # Other important settings
    remove_unused_columns=False,
    no_cuda=not torch.cuda.is_available(),
    fp16=torch.cuda.is_available(),
    overwrite_output_dir=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# Start training
print("Starting training...")
train_result = trainer.train()
print("Training completed!")

# Save final model
trainer.save_model("final_model")
print("Model saved to 'final_model' directory")

# Evaluate on test set
if small_eval_dataset is not None:
    print("Evaluating on validation set...")
    eval_results = trainer.evaluate()
    print(f"Validation results: {eval_results}")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1609: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
PyTorch: setting up devices
average_tokens_across_devices is True but world size is 1. Setting it to False automatically.
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Transformers version: 4.55.0
PyTorch version: 2.6.0+cu124
Using 1 GPU(s)
Steps per epoch: 250
Starting training...


***** Running training *****
  Num examples = 1,000
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 750
  Number of trainable parameters = 124,442,112
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mungaistephen (mungaistephen-murang-a-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


**Prepare labels for Token Classification**

The `AutoModelForTokenClassification` model expects a `labels` column in the dataset where each entry is a list of token-level class indices corresponding to the input tokens. You need to create this column based on your original sentiment span annotations.

For example, if your original labels are start and end indices of the sentiment span within the text, you would need to:
1. Align the character-level start and end indices with the tokenized input.
2. Create a list of labels for each token, where each label is an integer representing the class (e.g., 0 for 'O' - outside, 1 for 'B-sentiment' - beginning of sentiment span, 2 for 'I-sentiment' - inside of sentiment span). You might need different classes for different sentiment types (positive, negative, neutral).
3. Handle special tokens (like `[CLS]`, `[SEP]`, padding) by assigning them a special label value (e.g., -100) that the model can ignore during loss calculation.

In [10]:
# This is a placeholder cell. You need to implement the logic here
# to create the 'labels' column in your tokenized_datasets.

# Example: Function to align labels and create token-level labels
def align_labels_with_tokens(examples):
    tokenized_inputs = tokenizer(examples["text"], padding="max_length", truncation=True)
    labels = []
    for i, label in enumerate(examples["label"]): # Assuming 'label' is the sentiment class index
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # Create token-level labels based on the word ids and original sentiment span
        # This is a simplified example and needs to be adapted to your specific label format
        token_labels = []
        # You need to implement the logic to map original span labels to token labels
        # based on word_ids.
        # For now, let's create dummy labels (e.g., all 0s) to make the code runnable,
        # but this will not train the model correctly for sentiment extraction.
        for word_id in word_ids:
            if word_id is None:
                token_labels.append(-100) # Ignore special tokens
            else:
                # Replace this with your logic to assign the correct token label
                # based on the original sentiment span and the word_id
                token_labels.append(0) # Placeholder: Assign 0 for now

        labels.append(token_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the function to your dataset (replace with your actual dataset object)
# You might need to apply this to both train and test splits
# tokenized_datasets = dataset.map(align_labels_with_tokens, batched=True)

# After creating the 'labels' column, you should verify its structure and content.
# print(tokenized_datasets['train'][0]['labels'])

In [11]:
# Since the above cell is a placeholder and won't create correct labels for sentiment extraction,
# and to make the Trainer initialization runnable without the 'labels' error,
# let's temporarily add a dummy 'labels' column to the small datasets.
# **IMPORTANT:** This is only for making the code run and will NOT train the model correctly
# for the sentiment extraction task. You MUST replace this with the correct label preparation logic.

# Assuming the tokenized inputs have a certain sequence length (e.g., 512 from truncation=True in tokenize_function)
# You should get the actual sequence length from your tokenized dataset if it's different.
sequence_length = len(small_train_dataset[0]['input_ids']) # Get sequence length from actual data

def add_dummy_labels(examples):
    # Create dummy labels (e.g., all 0s or a random sequence) with the correct shape
    # The shape should be (batch_size, sequence_length) for token classification
    dummy_labels = [[0] * sequence_length for _ in range(len(examples['input_ids']))]
    examples['labels'] = dummy_labels
    return examples

# Apply the dummy label creation to your small datasets
# small_train_dataset = small_train_dataset.map(add_dummy_labels, batched=True)
# small_eval_dataset = small_eval_dataset.map(add_dummy_labels, batched=True)

# **Note:** The above dummy label creation is commented out because applying map again
# might overwrite previous tokenization results or cause issues if applied incorrectly.
# A better approach is to ensure the initial tokenization and labeling function
# correctly creates the 'labels' column.

# Assuming the 'labels' column is now correctly present in small_train_dataset and small_eval_dataset
# after running the appropriate label preparation logic.